In [59]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm
from torchvision.utils import save_image
import os

In [60]:
# 设置随机种子
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [61]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 定义生成器
class Generator(nn.Module):
    def __init__(self, latent_dim, img_shape):
        super(Generator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, int(torch.prod(torch.tensor(img_shape)))))
        self.img_shape = img_shape

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), *self.img_shape)
        return img

In [62]:
# 定义判别器
class Discriminator(nn.Module):
    def __init__(self, img_shape):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(torch.prod(torch.tensor(img_shape))), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
        self.img_shape = img_shape

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)
        return validity

In [63]:
# 定义训练函数
def train(num_epochs, generator, discriminator, dataloader, optimizer_G, optimizer_D, criterion, latent_dim, device):
    for epoch in range(num_epochs):
        for i, (imgs, _) in enumerate(tqdm(dataloader)):

            # 训练判别器
            real_imgs = imgs.to(device)
            optimizer_D.zero_grad()

            real_labels = torch.ones((real_imgs.size(0), 1)).to(device)
            fake_labels = torch.zeros((real_imgs.size(0), 1)).to(device)

            # 计算真实图像的损失
            real_outputs = discriminator(real_imgs)
            real_loss = criterion(real_outputs, real_labels)

            # 生成假图像并计算损失
            z = torch.randn((real_imgs.size(0), latent_dim)).to(device)
            fake_imgs = generator(z)
            fake_outputs = discriminator(fake_imgs.detach())
            fake_loss = criterion(fake_outputs, fake_labels)

            # 合并真实图像和假图像的损失
            d_loss = real_loss + fake_loss

            # 反向传播和优化
            d_loss.backward()
            optimizer_D.step()

            # 训练生成器
            optimizer_G.zero_grad()

            # 生成假图像并计算生成器损失
            fake_outputs = discriminator(fake_imgs)
            g_loss = criterion(fake_outputs, real_labels)

            # 反向传播和优化
            g_loss.backward()
            optimizer_G.step()

            if i % 100 == 0:
                print(f"[Epoch {epoch}/{num_epochs}] [Batch {i}/{len(dataloader)}] [D loss: {d_loss.item()}] [G loss: {g_loss.item()}]")

        # 保存生成的图像
        os.makedirs("images", exist_ok=True)  # 创建目录，如果目录已存在则忽略
        save_image(fake_imgs.data[:25], f"images/{epoch}.jpg", nrow=5, normalize=True)

In [64]:
# 设置超参数
latent_dim = 100
img_shape = (3, 64, 64)
lr = 0.0002
batch_size = 64
num_epochs = 10

In [65]:
# 创建生成器和判别器
generator = Generator(latent_dim, img_shape).to(device)
discriminator = Discriminator(img_shape).to(device)

In [66]:
# 定义损失函数和优化器
criterion = nn.BCELoss()
optimizer_G = Adam(generator.parameters(), lr=lr)
optimizer_D = Adam(discriminator.parameters(), lr=lr)

In [67]:
# 加载CelebA数据集
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [68]:
dataset = datasets.ImageFolder(root="../test10_AIGC/", transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)

In [70]:
# 训练GAN
train(1,generator, discriminator, dataloader, optimizer_G, optimizer_D, criterion, latent_dim, device)

  0%|          | 1/3166 [00:04<4:06:20,  4.67s/it]

[Epoch 0/1] [Batch 0/3166] [D loss: 0.2367042899131775] [G loss: 14.179183959960938]


  3%|▎         | 101/3166 [00:44<14:23,  3.55it/s]

[Epoch 0/1] [Batch 100/3166] [D loss: 0.36556166410446167] [G loss: 14.532881736755371]


  6%|▋         | 201/3166 [01:11<12:42,  3.89it/s]

...

 95%|█████████▍| 3001/3166 [13:21<00:39,  4.17it/s]

[Epoch 0/1] [Batch 3000/3166] [D loss: 0.5627950429916382] [G loss: 7.987661838531494]


 98%|█████████▊| 3101/3166 [13:46<00:16,  3.87it/s]

[Epoch 0/1] [Batch 3100/3166] [D loss: 0.5750430822372437] [G loss: 4.632171154022217]


100%|██████████| 3166/3166 [14:03<00:00,  3.75it/s]
